In [ ]:
!pip install langchain
!pip install -U langchain-community
!pip install gigachat
!pip install --upgrade --quiet  langchain-google-genai pillow
!pip install --upgrade --quiet  yandexcloud
!pip install yandex-chain
%pip install -qU langchain-openai

In [ ]:
# Настраиваем Api ключи
credentials_gigachat = ''
GOOGLE_API_KEY = ''
OPENAI_API_KEY = ''
secret_yandexgpt2 = ''
folder_id_yandex = ''

import getpass
import os
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

# Строим бенчмарк

In [28]:
from langchain.chat_models.gigachat import GigaChat
from langchain_google_genai import ChatGoogleGenerativeAI
from yandex_chain import ChatYandexGPT
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

import pandas as pd
import time

import re



def parse_gpt_output(gpt_output):
    """
    Парсит вывод GPT для получения массива с рейтингами моделей.

    :param gpt_output: Ответ GPT с рейтингом моделей.
    :return: Список с рейтингами моделей, где индекс соответствует номеру модели,
             а значение — место в рейтинге.
    """
    # Ищем массив в ответе GPT
    match = re.search(r'\[(.*?)\]', gpt_output)
    if match:
        # Преобразуем строку в список целых чисел
        pre_ranking = list(map(int, match.group(1).split(',')))
        ranking = [None]*len(pre_ranking)
        for ind, value in enumerate(pre_ranking):
          ranking[value - 1] = ind + 1
        return ranking
    else:
        raise ValueError("Не удалось найти массив с рейтингом в ответе GPT.")


class Benchmark:
    def __init__(self, model_names: list):
        self.models = {}

        if 'openai_gpt' not in model_names:
            raise KeyError('openai_gpt should be in an model_names')

        self.openai_gpt4_o = ChatOpenAI(
            model_name="gpt-4o",
            openai_api_key=OPENAI_API_KEY
        )
        self.models['openai_gpt4_o_mini'] = ChatOpenAI(
            model_name="gpt-4o-mini",
            openai_api_key=OPENAI_API_KEY
        )

        if 'gigachat' in model_names:
            self.models['giga'] = GigaChat(
                credentials=credentials_gigachat,
                model='GigaChat',
                verify_ssl_certs=False
            )
        if 'gemeni' in model_names:
            self.models['gemeni'] = ChatGoogleGenerativeAI(
                model="gemini-pro",
                convert_system_message_to_human=True
            )
        if 'yandexGPT' in model_names:
            self.models['yandexGPT'] = ChatYandexGPT(
                folder_id=folder_id_yandex,
                api_key=secret_yandexgpt2
            )

    def take_measurements(self, system_prompt, human_prompt):
        message = [
              SystemMessage(content=system_prompt),
              HumanMessage(content=human_prompt),
        ]

        times = []
        generations = []
        for num, model_name in enumerate(self.models.keys()):
            start = time.time()
            generation = self.models[model_name](message).content
            end = time.time()

            times.append(end - start)
            generations.append(generation)
        generation_quality = self.assess_the_quality(generations)

        result = pd.DataFrame({
            'model': list(self.models.keys()),
            'times': times,
            'generation quality': generation_quality,
            'generation': generations
        })
        return result

    def assess_the_quality(self, generations):
        prompt = ''
        for num_model, generation in enumerate(generations):
          prompt += f'Model number: {num_model + 1}\nGeneration: {generation}\n\n'

        system_prompt = """"Проанализируй генерации моделей по следующим критериям:

        1. **Полнота генерации**: Оцени, насколько хорошо каждая генерация покрывает все аспекты задания (завязка, основной конфликт, кульминация, развязка).
        2. **Увлекательность истории**: Оцени, насколько интересной и захватывающей кажется история.
        3. **Длина генерации**: Оцени, насколько длинна каждая генерация. Чем длиннее, тем ниже должна быть оценка.

        Для каждого критерия присвой оценку от 1 до 10, где 10 — это наилучший результат, а 1 — худший. Затем вычисли итоговый рейтинг для каждой модели, используя сумму оценок по всем критериям. Модель с наибольшей суммой должна занять 1-е место, а с наименьшей суммой — последнее место.

        Выведи результат в виде массива, где индекс соответствует номеру модели, а значение — это место в рейтинге (1 — лучшая генерация).

        Пример вывода:
        [2, 1, 3, 4] — означает, что модель 2 заняла первое место, модель 1 — второе место, модель 3 — третье место, модель 4 — четвертое место
        """
        message = [
              SystemMessage(content=system_prompt),
              HumanMessage(content=prompt)
        ]
        result = self.openai_gpt4_o(message).content
        return parse_gpt_output(result)

# Производим оценку

In [29]:
test_models = [
    'openai_gpt', 'gigachat', 'gemeni', 'yandexGPT'
]

system_prompt = """Ты — мощный мастер сказаний, и твоя задача — создать увлекательное приключение в мире Dungeons & Dragons, основываясь на предпочтениях пользователя.
  1. Создай историю, которая будет захватывающей, интересной и уникальной для каждого игрока, основываясь на их предпочтениях и желаниях.
  2. Обеспечь возможность для пользователя принимать решения, которые будут влиять на развитие истории. Включи несколько ключевых развилок или выборов, которые помогут игрокам управлять ходом событий.
  3. Учитывай предпочтения пользователя для создания нужной атмосферы и стиля. Добавь элементы, которые усилят погружение в мир игры.
  """
human_prompt = """Сформируй основную сюжетную линию, включающую:
   - **Завязка:** Описание начальной ситуации, с которой начинают приключение.
   - **Основной конфликт:** Главная проблема или угроза, с которой предстоит столкнуться игрокам.
   - **Кульминация:** Ключевой момент в истории, который приведет к развязке.
   - **Развязка:** Заключительная часть истории, которая подведет итог приключению. Должна зависеть от выбора пользователя (представить 2 варианта для концовки).
Предпочтения пользователя такие:
"Сгенерируй историю про волшебный сказочный лес с гоблинами и магами"
"""

bench = Benchmark(test_models)
df = bench.take_measurements(system_prompt, human_prompt)

/usr/local/lib/python3.10/dist-packages/langchain_google_genai/chat_models.py:381: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


In [30]:
df['LEN'] = df.generation.apply(lambda x: len(x))

In [44]:
df

,model,times,generation quality,generation,LEN
0,openai_gpt4_o_mini,6.180948,3.0,"### Сюжетная линия: ""Тайны Лунного Леса""\n\n##...",2837
1,giga,5.989570,1.0,"Завязка:\nВы - молодой маг, только что окончив...",1396
2,gemeni,8.077838,2.0,**Завязка:**\nВаша группа путешественников угл...,1300
3,yandexGPT,9.391127,4.0,#### **Приключение в волшебном лесу: тайны и о...,4495


In [51]:
df_sort = df.sort_values(by=['generation quality']).reset_index().drop(columns=['index'])
df_sort['times'] = df_sort['times'].round(2)
df_sort

,model,times,generation quality,generation,LEN
0,giga,5.99,1.0,"Завязка:\nВы - молодой маг, только что окончив...",1396
1,gemeni,8.08,2.0,**Завязка:**\nВаша группа путешественников угл...,1300
2,openai_gpt4_o_mini,6.18,3.0,"### Сюжетная линия: ""Тайны Лунного Леса""\n\n##...",2837
3,yandexGPT,9.39,4.0,#### **Приключение в волшебном лесу: тайны и о...,4495
